In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import scipy.io
import statsmodels.api as sm
from sklearn.linear_model import Lasso
from phot_funcs import *
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from hex_making_utils import *
#import fit_hex_grid
%matplotlib qt

In [ ]:
animal = "IM-"+input('rat: ') #i.e. IM-1006
#filepath = "/Volumes/Tim-1/Photometry Data/"+ animal + '/'
#filepath = "/Volumes/Tim/Photometry/"+ animal + '/'
filepath = "/Volumes/Tim K/Photometry Data/Triangle Maze/"+ animal + '/'
date = input('date: ') #format: mmddyyyy
ses_type = input('Block change = prob or barrier? ')
analcolor = 'green'# input('color of dLight?')
filecount = '0'# input('file suffix number: ')
# std used for gaussian smoothing of position data. In Weill building,
# recommend using sigma = 2
gausSigma = 0.1
usePosSmoothing = False

import phot_to_dframes

location = phot_to_dframes.location
sampledata = phot_to_dframes.sampledata
tridat = phot_to_dframes.tridat
visinds = sampledata.loc[sampledata.port.notnull(),:].index
fs = phot_to_dframes.Fs
datepath = savepath = phot_to_dframes.datepath

sampledata.loc[:,'x'] = sampledata.loc[:,'x'].interpolate()
sampledata.loc[:,'y'] = sampledata.loc[:,'y'].interpolate()

# if the video has been labeled using a cropped area, make sure to adjust position to account for the offset. x += 85

In [ ]:
sampledata.loc[:,"x"] = sampledata.loc[:,"x"].values+85

# adjust hex map to fit the video

In [ ]:
path = "/Volumes/Tim K/Photometry Data/Triangle Maze/"
#path = '/Users/berkelab/Desktop/Photometry Data/Triangle Maze/'
#path = '/Volumes/Seagate Backup Plus Drive/Data Backups/Photometry Data/Triangle Maze/'
#original_hexbins = np.load(path+'original_hexbins.npy',allow_pickle = True)
original_hexbins = np.load(path+'newMaze_hexbins.npy',allow_pickle = True)
#original_ids = np.load(path+'original_ids.npy',allow_pickle = True)
original_ids = np.load(path+'newMaze_ids.npy',allow_pickle = True)
#original_points = np.load(path+'original_points.npy',allow_pickle = True)
original_points = np.load(path+'new_points.npy',allow_pickle = True)

#run this to draw line. Can redraw as many times as needed
#draw circles in barriers, save to barriers. draw hexes 1,2,3,25 and save to new_points
import imageio
fname = datepath+'Behav_Vid'+filecount+'.avi'
vid = imageio.get_reader(fname, 'ffmpeg')

### transform_scatter will transform the scatter plot of hexes to fit the new map. the coordinates for each transformed centroid will be stored in the "new_coords" variable (in order of hex ID). You can access the transform parameters in the global variable "transformParams"

In [ ]:
transform_scatter(np.hstack([centroids,np.ones((len(centroids),1))]),vid.get_data(1000),rotate_center=[640/2,480/2])

### save the transform parameters for quick future access

In [ ]:
np.save(datepath,transformParams)

# identify barriers
## *be sure to click on the base of the barrier (where the overlaid hex is), not the top.*

In [ ]:
all_bars = []
bind = 0

In [ ]:
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    print("x = %d, y = %d" % (ix, iy))
    bar_coords.append((ix, iy))
print('max ind = ',len(blocks)-1)
b = blocks[bind]
ind = int(sampledata.loc[sampledata.block==b].index[int(phot_to_dframes.Fs*300)]/250*15) #5min into block
fig,ax = plt.subplots(figsize=(14,10))
ax.imshow(vid.get_data(ind), extent=[0, 640, 0, 480])
plt.scatter(new_coords[:,0],new_coords[:,1],marker='H',color='steelblue',alpha=0.5,s=1000)
#for h in range(len(hexmap.ids)):
#    plt.text(hexmap.new_hexes[h].centroid.x-10,hexmap.new_hexes[h].centroid.y+10,hexmap.ids[h],\
#            color='whitesmoke',fontweight='demibold')
bar_coords = []
# connect the click event handler
cid = fig.canvas.mpl_connect('button_press_event', onclick)

# show the plot
plt.show()

In [ ]:
all_bars.append(assign_to_hexagon(np.array(bar_coords)[:,0],np.array(bar_coords)[:,1],new_coords))
bind += 1

# convert position to hex ID

In [ ]:
hexes = assign_to_hexagon(sampledata.loc[sampledata.x.notnull(),'x'],sampledata.loc[sampledata.x.notnull(),'y'],new_coords)

## sanity check that position is getting binned in the correct hexes

In [ ]:
def plot_pos_and_its_hex(i):
    plt.clf()
    plt.scatter(sampledata.loc[sampledata.x.notnull(),'x'].values[i],sampledata.loc[sampledata.x.notnull(),'y'].values[i])
    plt.scatter(new_coords[:,0],new_coords[:,1],marker='H',color='steelblue',alpha=0.5,s=1000)
    plt.scatter(new_coords[hexes[i]-1,0],new_coords[hexes[i]-1,1],marker='H',color='darkred',alpha=0.5,s=1000)

In [ ]:
plt.figure()
interact(plot_pos_and_its_hex,i = widgets.IntSlider(
    value=20000,
    min=1,
    max=len(sampledata.loc[sampledata.x.notnull(),'x']),
    step=1,
    description='frame #:'))

In [ ]:
seshdat = Photrat(animal,date)
seshdat.df = sampledata

In [ ]:
seshdat.barIDs = np.array(barrier_ids)
#if barrier session, do for each block
seshdat.tmats = {b:[] for b in np.arange(len(blocks))}
if ses_type == 'barrier':
    segments = []
    branches = []
    for b in range(len(blocks)):
        hexmap.bar_ids = barrier_ids[b]
        hexmap.bar_ids = [float(f) for f in hexmap.bar_ids]
        mapfig = plot_hex_outline(size='lg',bars=hexmap.bar_ids)
        mapfig.savefig(datepath+"/hex_layout_block"+str(blocks[b])+".pdf")
        plt.title('animal: '+animal+'. Block '+str(blocks[b])+'. '+date)
        transition_matrix = hexmap.plot_sesh_tmat(blocks[b])
        seshdat.tmats[b] = transition_matrix
        plt.title('animal: '+animal+'. Block '+str(blocks[b])+'. '+date)
        plt.tight_layout()
        np.save(datepath+"/tmat_block_"+str(blocks[b]),transition_matrix)
    np.save(datepath+'/segments',segments)
else:
    hexmap.bar_ids = barrier_ids[0]
    hexmap.bar_ids = [float(f) for f in hexmap.bar_ids]
    mapfig = plot_hex_outline(size='lg',bars=hexmap.bar_ids)
    mapfig.savefig(datepath+"/hex_layout.pdf")
    plt.title('animal: '+animal+'. '+date)
    transition_matrix = hexmap.plot_sesh_tmat()
    seshdat.tmats[0] = transition_matrix
    plt.title('animal: '+animal+'. '+date)
    plt.tight_layout()
    np.save(datepath+"/tmat",transition_matrix)

# mark critical choice points (where rat has option to choose one port with left choice, and a different port with right choice)

In [ ]:
crit_cps = [[16]]
if ses_type == 'barrier':
    for b in range(len(blocks)):
        np.save(datepath+"/cps_block_"+str(blocks[b]),crit_cps[b])
else:
    np.save(datepath+"/cps",crit_cps[0])

# add hexes to dataframe

In [ ]:
sampledata.loc[:,'hexlabels']=-1
sampledata.loc[sampledata.x.notnull(),'hexlabels'] = hexes

# save final dataframe

In [ ]:
sampledata.to_csv(savepath+animal+'_'+date+'_'+'h_sampleframe.csv')